# Neural networks for anomaly detection

##### Debug mode

In [1]:
%load_ext autoreload
%autoreload 2

##### For relative imports

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

##### Notebook

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import mpld3
from interval import Interval
from observation import Observation
from constants import *
from datetime import timedelta
from tools import sequence_to_interval
plt.style.use('ggplot')
mpld3.enable_notebook()

PATH = "../../../Data/GMPP_IRSDI/"
reactor_sites = [site+tranche for site in ["A","B","C","D","E","F","G","H"] for tranche in ["1","2"]] + ["B3","B4","F3","F4"]

suffixes = [
"DEB1-1","DEB1-2","DEB1-3",#,"DEB1-4", # Débit de fuite au joint 1 (Gamme Large)
# "DEB2-1","DEB2-2","DEB2-3","DEB2-4", # Débit de fuite au joint 1 (Gamme Étroite)
# "DEB3-1","DEB3-2","DEB3-3","DEB3-4","DEB3-5", # Débit d'injection au joint
 "PUI-",  # Puissance thermique moyenne
 "PRE-"  # Pression
# "TEM1-", # Température ligne d'injection aux joints (en * Celsius) ### A rapprocher de DEB3
# "TEM2-", # Température fuites joint 1
# "TEM3-1","TEM3-2","TEM3-3","TEM3-4",# Température eau joint 1 - 051PO ### A rapprocher de DEB1 DEB2
# "VIT-1","VIT-2","VIT-3","VIT-4"# Vitesse de rotation
] 

obs_dict = {}

for reactor_site in reactor_sites:
    print("Loading observation "+reactor_site)
    obs_dict[reactor_site] = Observation(PATH,reactor_site,suffixes,verbose=0)



Loading observation A1
Loading observation A2
Loading observation B1
Loading observation B2
Loading observation C1
Loading observation C2
Loading observation D1
Loading observation D2
Loading observation E1
Loading observation E2
Loading observation F1
Loading observation F2
Loading observation G1
Loading observation G2
Loading observation H1
Loading observation H2
Loading observation B3
Loading observation B4
Loading observation F3
Loading observation F4


### Normal mode

In [4]:
# Re- exectute that everytime you have a 
# TypeError: super(type, obj): obj must be an instance or subtype of type
from scale import *
from feature import *
from tools import *
from score import *

<hr style="height:3px">

### Exploring PCA to find anomalies

In [5]:
from sklearn.decomposition import PCA

df = obs_dict["A1"].low_regime_intervals.split_between(obs_dict["A1"].full_concatenated_df, time=timedelta(days=3))[-1][deb1[0]]
data = df.values
data = np.array([data[t - half_window:t + half_window] for t in range(half_window, len(df) - half_window)])

def pca_on_data(n_components, data):
    pca = PCA(n_components)
    rolling_size=40
    half_window = int(rolling_size / 2)
    pca.fit(data)
    return pca

pca = pca_on_data(5, data)
score_5 = np.sum(np.power(pca.inverse_transform(pca.transform(data)) - data,2),axis=1)
pca = pca_on_data(15, data)
score_15 = np.sum(np.power(pca.inverse_transform(pca.transform(data)) - data,2),axis=1)
index = df.index[half_window:len(df) - half_window]
output_dataframe = np.concatenate([score_5,score_15]).reshape(-1,2)
dataframe = pd.DataFrame(index=index, data=output_dataframe, columns=["score_5","score_15"])

NameError: name 'half_window' is not defined

In [ ]:
t = 50 # Choose a sample to visualize
pca = pca_on_data(5, data)
plt.plot(data[t])
plt.plot(pca.inverse_transform(pca.transform(data[[t]])).ravel())
plt.title("DEB1 - A reconstructed sample (A1-2015)")

In [ ]:
t = 50 # Choose a sample to visualize
pca = pca_on_data(15, data)
plt.plot(data[t])
plt.plot(pca.inverse_transform(pca.transform(data[[t]])).ravel())
plt.title("DEB1 - A reconstructed sample (A1-2015)")

In [ ]:
fig, axes = plt.subplots(2,figsize=(10,6),sharex=True)
df.plot(ax = axes[0],color='b',title="DEB1")
(dataframe/30).plot(ax = axes[1],color=['k','r'])

Nb : 
- PCA is sensible to outliers
- PCA gives an indicator close to the variance
- PCA is better than variance in the sense that it can take multiple variables and learn from it

# Creating train / test datasets

**Required parameters**

- A resampling_scale for optional resampling.
- A length for the intervals. l_intervals.
- The stride between the intervals. Default to l_intervals. Min to 1.
- The proportion of test/train to evaluate if we don't overfit.
- A scale for fitting the data between 0 and 1, or normalizing it. 


**Required information to analyse performance**

- The method should return train and test, as well as all the corresponding timestamps.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def generate_train_test(obs_dict, target, scale, l_intervals, stride, test_size=0.):
    l = l_intervals // 2
    
    timestamps = pd.DatetimeIndex([])
    reactor_sites_array = []
    data = []
    
    for reactor_site, obs in obs_dict.items():
        list_df_resampled = obs.low_regime_intervals.split_between(scale.scale(obs.full_concatenated_df[target]),time=timedelta(days=3))
        for df in list_df_resampled:        
            indices = np.arange(l,df.shape[0]-l, stride)
            timestamps = timestamps.append(df.index[indices])
            data += [df.iloc[i-l:i+l].values[np.newaxis,:,:] for i in indices]
            reactor_sites_array+=[np.repeat(reactor_site,len(indices))]
    
    reactor_sites_array = np.concatenate(reactor_sites_array,axis=0)
    data = np.concatenate(data, axis=0)  
    
    not_nan_idx = ~np.isnan(data).any(axis=1).any(axis=1)
    data = data[not_nan_idx]
    timestamps = timestamps[not_nan_idx]
    reactor_sites_array = reactor_sites_array[not_nan_idx]
    
    data = (((data - data.min())/(data.max() - data.min()))*2 - 1)
    #train, test, train_timestamps, test_timestamps = train_test_split(data, timestamps)
    return data, timestamps, reactor_sites_array#train, test, timestamps

In [ ]:
target = [deb1[0],deb1[1]]
data, timestamps,reactor_sites_array = generate_train_test(obs_dict, target, HourScale(12), 50, 5, test_size=0.)
print(data.shape)
print("The reactor corresponding :",reactor_sites_array.shape)
print("The timestamps corresponding :",timestamps.shape)
x_train = data

### Training a simple neural network



In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.models import Model

x = Input(shape=x_train.shape[1:])

h = Flatten()(x)
h = Dense(64, activation='elu')(h)
h = Dense(16, use_bias=False)(h)
h = Dense(64, activation='elu')(h)

x_recons = Dense(x_train.shape[2]*x_train.shape[1])(h)
x_recons = Reshape(x_train.shape[1:])(x_recons)

mlt = Model(x, x_recons)
mlt.compile('adam', 'mse')

In [ ]:
epochs = 50
batch_size = 256

mlt = model_from_train(x_train)
mlt.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2
)

In [ ]:
x_train_reconstructed = mlt.predict(x_train)
print(x_train_reconstructed.shape)

### Visualizing the reconstruction

In [ ]:
fig, axes = plt.subplots(2,figsize=(5,6),sharex=True)
i = 2
axes[0].plot(x_train[i])
axes[1].plot(x_train_reconstructed[i])

In [ ]:
score = np.sum(np.sum((x_train_reconstructed - x_train)**2,axis = 1),axis=1) # All features combined
score_index = score.argsort()[::-1]
results = list(zip(timestamps[score_index],score[score_index],reactor_sites_array[score_index]))

In [ ]:
j = 40
timestamp, s, reactor_site = results[j]
print(s)
print(timestamp)
print(reactor_site)
fig, axes = plt.subplots(2,figsize=(10,6))
HourScale(12).scale(obs_dict[reactor_site].full_concatenated_df[target])[timestamp-timedelta(days=12):timestamp+timedelta(days=12)].plot(ax=axes[0])
axes[1].plot(x_train_reconstructed[score_index[j]])

In [ ]:
target = [deb1[0],deb1[1],pre[0],pui[0]]
data, timestamps_,reactor_sites_array = generate_train_test(obs_dict, target, HourScale(12), 50, 5, test_size=0.)
print(data.shape)
print("The reactor corresponding :",reactor_sites_array.shape)
print("The timestamps corresponding :",timestamps.shape)
x_train_ = data
epochs = 300
batch_size = 256

x = Input(shape=x_train_.shape[1:])

h = Flatten()(x)
h = Dense(128, activation='elu')(h)
h = Dense(64, use_bias=False)(h)
h = Dense(128, activation='elu')(h)

x_recons = Dense(x_train_.shape[2]*x_train_.shape[1])(h)
x_recons = Reshape(x_train_.shape[1:])(x_recons)

mlt = Model(x, x_recons)
mlt.compile('adam', 'mse')

mlt.fit(x_train_, x_train_,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2
)
x_train_reconstructed_ = mlt.predict(x_train_)

In [ ]:
score_ = np.sum(np.sum((x_train_reconstructed_ - x_train_)**2,axis = 1),axis=1) # All features combined
score_index_ = score_.argsort()[::-1]
results_ = list(zip(timestamps_[score_index_],score_[score_index_],reactor_sites_array[score_index_]))

In [ ]:
j = 5
timestamp, s, reactor_site = results_[j]
print(s)
print(timestamp)
print(reactor_site)

fig, axes = plt.subplots(2,3,figsize=(12,6),sharex=True)
i = score_index_[j]
axes[0,0].plot(x_train_[i,:,:2])
axes[0,1].plot(x_train_[i,:,2],color='k')
axes[0,2].plot(x_train_[i,:,3],color='g')
axes[1,0].plot(x_train_reconstructed_[i,:,:2])
axes[1,1].plot(x_train_reconstructed_[i,:,2],color='k')
axes[1,2].plot(x_train_reconstructed_[i,:,3],color='g')
plt.tight_layout()

## Learning better anomaly classifiers

Using neural networks to automatically learn our discriminative functions.

After this first train, these networks can be refined in a supervised fashion.

#### Exploring if the step function can be learned

In [ ]:
obs = obs_dict["A1"]
rolling_size=36
half_window = int(rolling_size / 2)-1
data = [] #data = np.concatenate(([[]for i in range(3)],[[1],[2],[3]]),axis=0)
for df in obs.low_regime_intervals.split_between(MinutesScale().scale(obs.full_concatenated_df), time=timedelta(days=3)):
    df = df[deb1[0]]
    data+= [np.array([df.values[t - half_window:t + half_window] for t in range(half_window, len(df) - half_window)])]
data = np.concatenate((data),axis=0)
print(data.shape)
#features_df.loc["step","6h"].df.dropna().shape

In [ ]:
from keras.layers import Input,Concatenate, MaxPooling1D,Conv1D,UpSampling1D,Input, Dense, Lambda, Layer, Dropout, Conv1D, BatchNormalization, Activation, Reshape,Flatten
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras import optimizers

x = Input(shape=x_train.shape[1:])

# Neural Network parameters
intermediate_dim = 256
latent_dim = 10 

# Encoder
h = x
h = Dense(intermediate_dim, activation='relu')(h)
h = Dense(intermediate_dim//2, activation='relu')(h)
y = h
step_learner = Model(x,y)
step_learner.compile(optimizer='rmsprop', loss='mean_squared_error')

epochs = 30
batch_size = 256

vae.fit(x_train, score_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test,score_test),
        verbose=0
)